Field() and StringConstraints()

In [1]:
from typing import Annotated
from pydantic import BaseModel, Field, StringConstraints, StrictStr, EmailStr

In [2]:
!pip install pydantic[email]

In [3]:
NonEmptyList = Annotated[list[StrictStr], Field(..., min_length=1), StringConstraints(min_length=3)]
StrippedString = Annotated[StrictStr, StringConstraints(strip_whitespace=True)]

In [4]:
class ValidationTst(BaseModel):
    variable: NonEmptyList = None
    var: StrippedString = Field(repr=False)

In [5]:
model = ValidationTst(var="      ")
model.var

''

In [6]:
LowerAlphaNumericStr = Annotated[StrictStr, StringConstraints(pattern=r"^[a-z0-9_]+$")]
SlackNameField = Annotated[str, StringConstraints(pattern=r"^[#@][a-z0-9_]+$")]

In [7]:
class ValidationTst2(BaseModel):
    name: SlackNameField

model = ValidationTst2(name="#")

ValidationError: 1 validation error for ValidationTst2
name
  String should match pattern '^[#@][a-z0-9_]+$' [type=string_pattern_mismatch, input_value='#', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_pattern_mismatch

In [11]:
Str2 = Annotated[StrictStr, StringConstraints(max_length=2)]
LowerEmailStr = Annotated[Str2, EmailStr, StringConstraints(to_lower=True, max_length=2)]

In [12]:
class Hello(BaseModel):
    e: LowerEmailStr

In [14]:
Hello(e="")

ValidationError: 1 validation error for Hello
e
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='', input_type=str]

In [ ]:
class Comp(BaseModel):
    c1: int
class DataOrder(BaseModel):
    c: list[Comp]

In [ ]:
d1 = DataOrder(c=[Comp(c1=1)])
d2 = {"c": [{"c1": 23}]}

In [ ]:
t1 = DataOrder.model_construct(**d1.model_dump())
t1

In [ ]:
t2 = DataOrder.model_construct(**d2)
t2

In [ ]:
from typing import Annotated
from decimal import Decimal
NonNegativeDecimal = Annotated[Decimal, Field(ge=0, decimal_places=2, allow_inf_nan=False)]

In [ ]:
class Cost(BaseModel):
    value: NonNegativeDecimal

In [ ]:
Cost(value=23.98)

In [ ]:
import math
def round_to_base(num: Decimal | float, base: int) -> int:
    """
    Returns a number ceil-rounded to a multiple of provided base

    :param num: the number to round
    :param base: the base to round to
    :return: the rounded number

    >>> round_to_base(61, 4)
    64
    >>> round_to_base(3.5, 1)
    4
    >>> round_to_base(3, 10)
    10
    """
    assert base > 0
    return base * math.ceil(num / base)

In [ ]:
round_to_base(1, 0.01)

In [ ]:
!pip install -U pydantic

In [ ]:
from pydantic import BaseModel, AfterValidator
from typing import Annotated
import json


class User(BaseModel):
    name: str
    id: int

class Test(BaseModel):
    user: dict[str, Annotated[User, AfterValidator(lambda m: m.model_dump())]]


u = User(id=1, name="Y")
t = Test(user={"1": u})
d = {"test": t.user}
json.dumps(d)

'{"test": {"1": {"name": "Y", "id": 1}}}'

In [15]:
from pydantic import AfterValidator, StringConstraints

UpperStr = Annotated[str, AfterValidator(lambda v: v.upper())]
UpperStr2 = Annotated[str, StringConstraints(to_upper=True)]
class TestClass(BaseModel):
    lst: list[UpperStr] | None = None
    lst2: list[UpperStr2] | None = None

TestClass(lst=["a", "b"], lst2=["c", "d"]).model_dump()

{'lst': ['A', 'B'], 'lst2': ['C', 'D']}